In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt
import time

sys.path.insert(0, "/home/ylu/project/Benchmark_alignment_methods/scripts/methods")
from my_STAlign import stalign_align
sys.path.insert(0, "/home/ylu/project")
from utils import *
sys.path.insert(0, "/home/ylu/project/STalign")
from STalign import STalign
%load_ext autoreload
%autoreload 2

In [2]:
data_folder = "../../data/STARMapPlus_nonrigid_simulation_different_distort/"
results_folder = "./nonrigid_different_distort_results/STAlign_wo_prealign"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [3]:
slice_name = ['A','B','C','D']
tps_slice_name = ['A*','B*','C*','D*']
# i = 3
# data_name = "sagittal{}".format(i)
i = 10
data_name = "well_{}".format(i)
s = ad.read_h5ad('../../data/STARMapPlus/{}.h5ad'.format(data_name))
norm_scale = np.linalg.norm(s.obsm['spatial'].max(0) - s.obsm['spatial'].min(0))
del s
j = 1
slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))
repeat = 10

In [ ]:
for l, no in enumerate(np.arange(0,0.22,0.02)):
    alignment_results = []
    for k in range(repeat):
        # if l < 2:
        #     continue
        slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
        slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()

        # R, t = solve_RT_by_correspondence(slice2.obsm['spatial'], slice2.obsm['prealign_spatial'])
        # slice2.obsm['prealign_spatial'] = slice2.obsm['prealign_spatial'] @ R.T + t
        
        # align slices
        align_slices = stalign_align(
            models = [slice1, slice2],
            spatial_key = 'prealign_spatial',
            key_added = 'align_spatial',
            device = "cuda:0",
            scale=0.3,
            zero_mean=True,
        )
        torch.cuda.empty_cache()
        mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
        mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
        print(f'------iteration: {l}----repeat: {k}-----mae: {mae}---------')
        
        alignment_results.append([mae, mae_var, mse, mse_var, data_name, no, l, k])

    np.save(os.path.join(results_folder, data_name + "_STAlign_differ_distort_{}.npy".format(l)), alignment_results, allow_pickle=True)

/home/ylu/project/STalign/STalign/STalign.py:1044: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  L = torch.tensor(L,device=device,dtype=dtype,requires_grad=True)
/home/ylu/project/STalign/STalign/STalign.py:1045: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  T = torch.tensor(T,device=device,dtype=dtype,requires_grad=True)
/home/ylu/anaconda3/envs/STAlign/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


------iteration: 0----repeat: 0-----mae: 0.4321505898351903---------


/home/ylu/anaconda3/envs/STAlign/lib/python3.9/site-packages/torch/utils/_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


------iteration: 0----repeat: 1-----mae: 0.38031038818914736---------
------iteration: 0----repeat: 2-----mae: 0.42572739172617197---------
------iteration: 0----repeat: 3-----mae: 0.41628389074724026---------
------iteration: 0----repeat: 4-----mae: 0.06508009825273448---------
------iteration: 0----repeat: 5-----mae: 0.4583892364252176---------
------iteration: 0----repeat: 6-----mae: 0.17455505392534104---------
------iteration: 0----repeat: 7-----mae: 0.33708301521655654---------
------iteration: 0----repeat: 8-----mae: 0.21950149036288985---------
------iteration: 0----repeat: 9-----mae: 0.1814833750807202---------
------iteration: 1----repeat: 0-----mae: 0.1646309363229182---------
------iteration: 1----repeat: 1-----mae: 0.4607932677632544---------
------iteration: 1----repeat: 2-----mae: 0.458974990485923---------
------iteration: 1----repeat: 3-----mae: 0.4579219598167115---------
------iteration: 1----repeat: 4-----mae: 0.06776777291769191---------
------iteration: 1----repea

In [ ]:
for l, no in enumerate(np.arange(0,0.22,0.02)):
    alignment_results = []
    for k in range(repeat):
        slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
        slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()
        align_slices, _, _ = paste_align_ref(
        models=[slice1.copy(), slice2.copy()],
            spatial_key="prealign_spatial",
            key_added="align_spatial",
            device="1",
            verbose=False,
            n_sampling=5000,
            sampling_method="random",
        )
        torch.cuda.empty_cache()
        mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
        mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
        
        alignment_results.append([mae, mae_var, mse, mse_var, data_name, no, l, k])

    np.save(os.path.join(results_folder, data_name + "_SPACEL_differ_distort_{}.npy".format(l)), alignment_results, allow_pickle=True)

In [ ]:
# sagittal_morpho_results = {}
spatial_key="prealign_spatial"
key_added="align_spatial"
anno_key = 'Main_molecular_tissue_region'
slice_name = ['A','B','C','D']
tps_slice_name = ['A*','B*','C*','D*']
# for i in range(16):
i = 3
# data_name = "well_{:02d}".format(i)
data_name = "sagittal{}".format(i)
s = ad.read_h5ad('../../data/STARMapPlus/{}.h5ad'.format(data_name))
norm_scale = np.linalg.norm(s.obsm['spatial'].max(0) - s.obsm['spatial'].min(0))
del s

for j in range(3):
    slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
    slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))

    for l, no in enumerate(np.arange(0,0.22,0.02)):
        alignment_results = []
        for k in range(repeat):
            slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
            slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()

            sampling_num = 5000
            sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
            sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
            sub_slice1 = slice1[sampline_idx1,:]
            sub_slice2 = slice2[sampline_idx2,:]
    
            # align slices
            align_slices = spacel_align(
                models=[sub_slice1, sub_slice2],
                spatial_key=spatial_key,
                key_added=key_added,
                anno_key = anno_key,
            )
        
            mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
            mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
    
            alignment_results.append([mae, mae_var, mse, mse_var, data_name, slice_name[j]+tps_slice_name[j+1]])
    
for j in range(3):
    slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))
    slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
    slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
    slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_0'].copy()

    # align slices
    align_slices = spacel_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        anno_key = anno_key,
    )

    mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
    mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
    
    alignment_results.append([mae, mae_var, mse, mse_var, data_name, slice_name[j+1]+tps_slice_name[j]])

np.save(os.path.join(results_folder, data_name + "_SPACEL.npy"), alignment_results, allow_pickle=True)